In [ ]:
# 세션 유지 코드
from IPython.display import clear_output
import threading, time

def keep_alive():
    for i in range(100000):
        time.sleep(60)
        clear_output(wait=True)
        print(f"Ping {i} ⏱️")

threading.Thread(target=keep_alive).start()

In [ ]:
# ===================================================
# 0. Install dependencies (Colab)
# ===================================================
!pip install transformers datasets sentence-transformers accelerate peft bitsandbytes trl --quiet

import torch, re
import pandas as pd
from datasets import load_dataset, Dataset
from transformers import (AutoTokenizer, AutoModelForSeq2SeqLM, BitsAndBytesConfig,
                          TrainingArguments, DataCollatorWithPadding)
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer

In [ ]:
# ==========================================
# 1. Load Dataset (Math binary classification)
# ==========================================
print("🔄 Loading dataset...")

from google.colab import drive
drive.mount('/content/drive')

dataset = load_from_disk("/content/drive/MyDrive/Gemma_FineTuning/essay_eval_multiclass_2000")

In [ ]:
# ===================================================
# 2. Load Gemma Model + LoRA Setup
# ===================================================

from huggingface_hub import login
login(token="YOUR_HF_TOKEN")

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

gemma_model_id = "google/gemma-2b-it"
tokenizer = AutoTokenizer.from_pretrained(gemma_model_id)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

base_model = AutoModelForSeq2SeqLM.from_pretrained(
    gemma_model_id,
    quantization_config=quant_config,
    device_map="auto"
)

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q", "v"],
    lora_dropout=0.1,
    bias="none",
    task_type="SEQ_2_SEQ_LM"
)

model = get_peft_model(base_model, lora_config)

In [ ]:
# ===================================================
# 3. Tokenization & Trainer Setup
# ===================================================
def tokenize(example):
    return tokenizer(example["input"], truncation=True, padding="max_length", max_length=512)

tokenized_data = data.map(tokenize, batched=True)

training_args = TrainingArguments(
    output_dir="./gemma-essay-lora",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=2,
    logging_steps=25,
    save_steps=500,
    learning_rate=2e-4,
    lr_scheduler_type="constant",
    bf16=True,
    optim="paged_adamw_8bit",
    report_to=None,
)

trainer = SFTTrainer(
    model=model,
    train_dataset=tokenized_data,
    args=training_args,
    peft_config=lora_config,
    data_collator=DataCollatorWithPadding(tokenizer)
)

In [ ]:
# ===================================================
# 4. Start Training
# ===================================================
print("🚀 Starting LoRA fine-tuning on essay dataset...")
trainer.train()

In [ ]:
# ==========================================
# 5. Save to Drive & Hugging Face
# ==========================================

save_path = "/content/drive/MyDrive/Gemma_FineTuning/essay_eval_multiclass_2000"
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

# Hugging Face Upload
repo_id = "LeannaJ/essay_evaluation"
model.push_to_hub(repo_id)
tokenizer.push_to_hub(repo_id)
print(f"✅ Hugging Face에 업로드 완료: https://huggingface.co/{repo_id}")